In [1]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [2]:
import sys, random
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
# from my_code import pytorch_model as m

In [3]:
def time_left(time_start, epoch, i, len_x):
    time_left = (time.time() - time_start) * (100*len_x//32 / (epoch*len_x//32 + i + 1) - 1)
    hours = int(time_left // 3600)
    minutes = int((time_left - hours * 3600) // 60)
    seconds = int(time_left - hours * 3600 - minutes * 60)
    return hours, minutes, seconds

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import time


# Define your VAE architecture
class VAE(nn.Module):
    def __init__(
                    self, 
                    emb_dim, 
                    latent_dim, 
                    hidden_dim=1024,
                    num_layers=5,
                    rnn_type='LSTM'
                ):                
        super(VAE, self).__init__()

        self.emb_dim = emb_dim
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn_type = rnn_type

        rnn_fn = {'LSTM': nn.LSTM, 'GRU': nn.GRU, 'RNN': nn.RNN}[rnn_type]

        #--- ENCODER ---#
        self.embedding = nn.Embedding(19, emb_dim)
        self.enc_rnn =  rnn_fn(
            input_size=emb_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers,  
            batch_first=True,                               
        )
        self.enc_dense = nn.Sequential(nn.Linear(hidden_dim*num_layers, 64), nn.ReLU())
        self.enc_dense_mu = nn.Linear(64, latent_dim)
        self.enc_dense_logvar = nn.Linear(64, latent_dim)


        #--- DECODER ---#
        self.dec_dense_init = nn.Sequential(
            nn.Linear(latent_dim, 64), nn.ReLU(),
            nn.Linear(64, hidden_dim * num_layers)
        )
        self.dec_dense_emb = nn.Linear(1, emb_dim)
        self.dec_rnn = rnn_fn(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        self.dec_dense_out = nn.Linear(hidden_dim, 12)

    def encoder(self, x):
        x = self.embedding(x)
        _, t = self.enc_rnn(x)
        h = t[0] if self.rnn_type == 'LSTM' else t
        x = h.view(-1, self.hidden_dim*self.num_layers)
        x = self.enc_dense(x)
        mu = self.enc_dense_mu(x)
        logvar = self.enc_dense_logvar(x)
        return mu, logvar
    
    def decoder(self, z):
        h0 = self.dec_dense_init(z).view(self.num_layers, -1, self.hidden_dim)
        t = (h0, torch.zeros_like(h0)) if self.rnn_type == 'LSTM' else h0
        x = self.dec_dense_emb(z.view(-1, self.latent_dim, 1))
        x, _ = self.dec_rnn(x, t)
        x = self.dec_dense_out(x[:, -1, :])
        return x

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std)
        z = mu + epsilon * std
        return z 

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

# Define your loss function
def loss_function(reconstructed, x, mu, logvar):
    reconstruction_loss = F.mse_loss(reconstructed, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  
    return reconstruction_loss + kl_divergence

# Initialize your VAE
device = "cuda:1"
emb_dim = 10 
latent_dim = 50
vae = VAE(
    emb_dim, 
    latent_dim,    
    hidden_dim=1024,
    num_layers=5,
    rnn_type='RNN',
).to(device)

# Define your optimizer
optimizer = optim.Adam(vae.parameters(), lr=0.001)

# data
data = c.Data.load(initial_path=initial_path, file_name='PET_SCORES_12_Numbers_int')
x = data.x_train.int().to(device)
x_test = data.x_test.int().to(device)
y = data.y_train.float().to(device)
y_test = data.y_test.float().to(device)

x_float = x.float()
x_test_float = x_test.float()

time_start = time.time()
loss_list = []
for epoch in range(100):
    ind = torch.randperm(len(x))
    x, x_float, y = x[ind], x_float[ind], y[ind]
    len_x = len(x)
    for i, (x_batch, x_batch_float, y_batch) in enumerate(zip(x.split(32), x_float.split(32), y.split(32))):

        #train
        optimizer.zero_grad()
        reconstructed, mu, logvar = vae(x_batch)
        loss = loss_function(reconstructed, x_batch_float, mu, logvar)
        loss.backward()
        optimizer.step()

        #time and print
        h, m, s = time_left(time_start, epoch, i, len_x)

        #loss
        loss = loss.item()
        loss_list.append(loss)

        #print
        print(f'Epoch {epoch}, batch {i}/{len_x//32}, loss={loss:.4f}, time left={h}h {m}m {s}s                                    ', end='\r')

    # print loss on test set
    loss_last_epoch = sum(loss_list[-len_x//32:]) / (len_x//32)
    print(f'Epoch {epoch}, loss={loss_last_epoch:.6f},                                                                                           ', end='\n')


# After training, you can generate new sequences
# with torch.no_grad():
#     z_sample = torch.randn(1, latent_dim)
#     generated_sequence = vae.decoder(z_sample)


Epoch 0, loss=25.572827,                                                                                           
Epoch 1, loss=25.516971,                                                                                           
Epoch 2, loss=14647723565.375923,                                                                                           
Epoch 3, loss=44532.028903,                                                                                           
Epoch 4, loss=344.018009,                                                                                           
Epoch 5, loss=1260.876336,                                                                                           
Epoch 6, loss=142.488106,                                                                                           
Epoch 7, loss=164029517.726809,                                                                                           
Epoch 8, loss=nan,                               

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import time


# Define your VAE architecture
class VAE(nn.Module):
    def __init__(self, emb_dim, latent_dim):                
        super(VAE, self).__init__()

        self.emb_dim = emb_dim
        self.latent_dim = latent_dim

        #--- ENCODER ---#
        self.embedding = nn.Embedding(19, emb_dim)
        self.enc_lstm1 = nn.LSTM(emb_dim, 64, batch_first=True, num_layers=1)
        self.enc_lstm2 = nn.LSTM(64, 64, batch_first=True, num_layers=3)
        self.enc_dense = nn.Sequential(nn.Linear(64, 64), nn.ReLU())
        self.enc_dense_mu = nn.Linear(64, latent_dim)
        self.enc_dense_logvar = nn.Linear(64, latent_dim)


        #--- DECODER ---#
        self.dec_dense1 = nn.Sequential(nn.Linear(latent_dim, 64), nn.ReLU(), nn.Linear(64, 12*64))
        self.dec_lstm = nn.LSTM(64, 64, batch_first=True, num_layers=5)
        self.dec_dense2 = nn.Linear(64, 12)
        self.dec_dense3 = nn.Linear(12, 12)


    def decoder(self, z):
        z = self.dec_dense1(z)
        z = z.view(-1, 12, 64)
        z, _ = self.dec_lstm(z)
        z = self.dec_dense2(z[:, -1, :])
        z = nn.ReLU()(z)
        z = self.dec_dense3(z)
        return z
        
    def encoder(self, x):
        x = self.embedding(x)
        x, _ = self.enc_lstm1(x)
        x, _ = self.enc_lstm2(x)
        x = self.enc_dense(x[:, -1, :])
        mu = self.enc_dense_mu(x)
        logvar = self.enc_dense_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std)
        z = mu + epsilon * std
        return z 

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

# Define your loss function
def loss_function(reconstructed, x, mu, logvar):
    reconstruction_loss = F.mse_loss(reconstructed, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  
    return reconstruction_loss + kl_divergence

# Initialize your VAE
device = "cuda:1"
emb_dim = 100 
latent_dim = 12
vae = VAE(emb_dim, latent_dim).to(device)

# Define your optimizer
optimizer = optim.Adam(vae.parameters(), lr=0.001)

# data
data = c.Data.load(initial_path=initial_path, file_name='PET_SCORES_12_Numbers_int')
x = data.x_train.int().to(device)
x_test = data.x_test.int().to(device)
y = data.y_train.float().to(device)
y_test = data.y_test.float().to(device)

x_float = x.float()
x_test_float = x_test.float()

time_start = time.time()
loss_list = []
for epoch in range(100):
    ind = torch.randperm(len(x))
    x, x_float, y = x[ind], x_float[ind], y[ind]
    len_x = len(x)
    for i, (x_batch, x_batch_float, y_batch) in enumerate(zip(x.split(32), x_float.split(32), y.split(32))):

        #train
        optimizer.zero_grad()
        reconstructed, mu, logvar = vae(x_batch)
        loss = loss_function(reconstructed, x_batch_float, mu, logvar)
        loss.backward()
        optimizer.step()

        #time and print
        h, m, s = time_left(time_start, epoch, i, len_x)

        #loss
        loss = loss.item()
        loss_list.append(loss)

        #print
        print(f'Epoch {epoch}, batch {i}/{len_x//32}, loss={loss:.4f}, time left={h}h {m}m {s}s                                    ', end='\r')

    # print loss on test set
    loss_last_epoch = sum(loss_list[-len_x//32:]) / (len_x//32)
    print(f'Epoch {epoch}, loss={loss_last_epoch:.6f},                                                                                           ', end='\n')


# After training, you can generate new sequences
# with torch.no_grad():
#     z_sample = torch.randn(1, latent_dim)
#     generated_sequence = vae.decoder(z_sample)


In [ ]:
model.eval()
model(data_x_tensor[0:1])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# VAE Model Definition
class VAE(nn.Module):
    def __init__(self, emb_dim, latent_dim):                
        super(VAE, self).__init__()

        self.emb_dim = emb_dim
        self.latent_dim = latent_dim

        #--- ENCODER ---#
        self.embedding = nn.Embedding(19, emb_dim)
        self.enc_lstm = nn.LSTM(emb_dim, 64, batch_first=True, num_layers=3)
        self.enc_dense = nn.Sequential(nn.Linear(64, 64), nn.ReLU())
        self.enc_dense_mu = nn.Linear(64, latent_dim)
        self.enc_dense_logvar = nn.Linear(64, latent_dim)

        # #--- DECODER ---#
        self.dec_dense1 = nn.Sequential(nn.Linear(latent_dim, 64), nn.ReLU(), nn.Linear(64, 64))
        self.dec_lstm = nn.LSTM(64, 64, batch_first=True, num_layers=5)
        self.dec_dense2 = nn.Sequential(nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, 19*12), nn.ReLU(), nn.Linear(19*12, 19*12), nn.ReLU(), nn.Linear(19*12, 19*12))

    def decoder(self, z):
        z = self.dec_dense1(z)
        z = z.unsqueeze(1).repeat(1, 64, 1)
        z, _ = self.dec_lstm(z)
        z = z[:, -1, :]
        z = self.dec_dense2(z)
        z = z.view(-1, 12, 19)
        return F.softmax(z, dim=-1)
        
    def encoder(self, x):
        x = self.embedding(x)
        x, (h, c) = self.enc_lstm(x)
        x = h[-1, :, :]
        x = self.enc_dense(x)
        mu = self.enc_dense_mu(x)
        logvar = nn.ReLU()(self.enc_dense_logvar(x))
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std)
        z = mu + epsilon * std
        return z 

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

# Loss Function
def vae_loss(reconstructed, original, mu, logvar):
    flattened_original = original.view(-1)
    flattened_reconstructed = reconstructed.view(-1, 19)
    recon_loss = F.cross_entropy(flattened_reconstructed, flattened_original, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    total_loss = recon_loss + kl_divergence
    return total_loss


device = "cuda:1"

# Assume data_x is a numpy array, convert to tensor
data = c.Data.load(initial_path=initial_path, file_name='PET_SCORES_12_Numbers_int')
data_x_tensor = data.x_train.long().to(device)

# DataLoader
dataset = TensorDataset(data_x_tensor)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model and Optimizer Initialization
emb_dim = 20  
latent_dim = 12  
model = VAE(emb_dim, latent_dim).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training Loop
time_start = time.time()
loss_list = []
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    train_loss = 0
    for i, batch_data in enumerate(data_loader):

        #train
        batch_data = batch_data[0]
        
        optimizer.zero_grad()
        reconstructed, mu, logvar = model(batch_data)
        loss = vae_loss(reconstructed, batch_data, mu, logvar)
        
        loss.backward()
        optimizer.step()

        #time and print
        h, m, s = time_left(time_start, epoch, i, len_x)

        #loss
        loss = loss.item()
        loss_list.append(loss)

        #print
        print(f'Epoch {epoch+1}, batch {i}/{len(data_loader)}, loss={loss:.4f}, time left={h}h {m}m {s}s                                    ', end='\r')

    # print loss on test set
    loss_last_epoch = sum(loss_list[-len(data_loader):]) / len(data_loader)
    print(f'Epoch {epoch+1}, loss={loss_last_epoch:.6f},                                                                                           ', end='\n')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class one_hot_encode(nn.Module):
    def __init__(self, num_classes=19):
        super(one_hot_encode, self).__init__()
        self.num_classes = num_classes

    def forward(self, tensor):
        one_hot = torch.zeros(tensor.size(0), tensor.size(1), self.num_classes, device=tensor.device)
        one_hot.scatter_(2, tensor.unsqueeze(-1).long(), 1)
        return one_hot

# VAE Model Definition
class VAE(nn.Module):
    def __init__(self, latent_dim):                
        super(VAE, self).__init__()
        self.latent_dim = latent_dim

        #--- ENCODER ---#
        self.enc = nn.Sequential(
            one_hot_encode(19), nn.Flatten(), 
            nn.Linear(19*12, 64), nn.LeakyReLU(), 
            nn.Linear(64, 64), nn.LeakyReLU()
        )
        self.enc_dense_mu = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, latent_dim))
        self.enc_dense_logvar = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, latent_dim))

        # #--- DECODER ---#
        self.dec = nn.Sequential(
            nn.Linear(latent_dim, 50), nn.LeakyReLU(), 
            nn.Linear(50, 100), nn.LeakyReLU(),
            nn.Linear(100, 200), nn.LeakyReLU(),
            nn.Linear(200, 19*12)
        )
        
    def encoder(self, x):
        x = self.enc(x)
        mu = self.enc_dense_mu(x)
        logvar = self.enc_dense_logvar(x)
        return mu, logvar

    def decoder(self, z):
        z = self.dec(z)
        z = z.view(-1, 12, 19)
        return F.softmax(z, dim=-1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(std)
        z = mu + epsilon * std
        return z 

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

# Loss Function
def vae_loss(reconstructed, original, mu, logvar):
    flattened_original = original.view(-1)
    flattened_reconstructed = reconstructed.view(-1, 19)
    recon_loss = F.cross_entropy(flattened_reconstructed, flattened_original) #, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    total_loss = recon_loss + kl_divergence
    return total_loss


# device = "cuda:1"
device = "cpu"

# Assume data_x is a numpy array, convert to tensor
data = c.Data.load(initial_path=initial_path, file_name='PET_SCORES_12_Numbers_int')
data_x_tensor = data.x_train.long().to(device)

# DataLoader
dataset = TensorDataset(data_x_tensor)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model and Optimizer Initialization 
latent_dim = 6  
model = VAE(latent_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Loop
time_start = time.time()
loss_list = []
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    train_loss = 0
    for i, batch_data in enumerate(data_loader):

        #train
        batch_data = batch_data[0]
        
        optimizer.zero_grad()
        reconstructed, mu, logvar = model(batch_data)
        loss = vae_loss(reconstructed, batch_data, mu, logvar)
        
        loss.backward()
        optimizer.step()

        #time and print
        h, m, s = time_left(time_start, epoch, i, len_x)

        #loss
        loss = loss.item()
        loss_list.append(loss)

        #print
        print(f'Epoch {epoch+1}, batch {i}/{len(data_loader)}, loss={loss:.4f}, time left={h}h {m}m {s}s                                    ', end='\r')

    # print loss on test set
    loss_last_epoch = sum(loss_list[-len(data_loader):]) / len(data_loader)
    print(f'Epoch {epoch+1}, loss={loss_last_epoch:.6f},                                                                                           ', end='\n')


In [ ]:
x = torch.argmax(model.to('cuda')(data.x_test[160:165].to('cuda'))[0], dim=-1)
#round to closest integer
x 

In [ ]:
data.x_test[162:165].to('cuda')